In [1]:
import numpy as np
import pandas as pd
import redis

In [18]:
# redis-12025.c85.us-east-1-2.ec2.cloud.redislabs.com:12025
hostname='redis-12025.c85.us-east-1-2.ec2.cloud.redislabs.com'
port=12025
password='LhNFHs1KTyB9C1MWktmKlu2oP8iSr6C6'
r=redis.StrictRedis(host=hostname,
                    port=port,
                    password=password)

In [3]:
r.ping()

True

In [4]:
# r.keys()

[]

In [7]:
# load numpy zip data
f=np.load('DataFrame_students_teachers.npz',allow_pickle=True)
x_values=f['arr_0']
col=f['arr_1']
df=pd.DataFrame(x_values,columns=col)
df

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.4324317, 1.074975, -0.12851736, -1.833421, ..."
1,Barack Obama,Teacher,"[0.94846946, -0.5583677, 0.09491567, -1.279613..."
2,Chris Evans,Student,"[0.72002447, -1.4682528, -0.77743137, -1.45999..."
3,Morgan Freeman,Teacher,"[0.48562637, -0.61965686, -0.4072629, 1.068597..."
4,Scarlett Johansson,Student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732..."


In [10]:
# We have to store data in redis in hash table 
'''there cannot be 2 keys for a value therefore we will
combine name and role into 1 column'''

df['name_role']=df['Name']+'@'+df['Role'] 

In [11]:
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,Student,"[0.4324317, 1.074975, -0.12851736, -1.833421, ...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.94846946, -0.5583677, 0.09491567, -1.279613...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.72002447, -1.4682528, -0.77743137, -1.45999...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.48562637, -0.61965686, -0.4072629, 1.068597...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732...",Scarlett Johansson@Student


In [12]:
records=df[['name_role','Facial_Features']].to_dict(orient='records')
len(records)

5

In [15]:
# records

# Save data  in redis hashes

In [20]:
#cannot store facial features directly i.e array we need to conver it to bytes
for record in records:
    name_role=record['name_role']
    vector=record['Facial_Features']
    
    v_bytes=vector.tobytes()
    r.hset(name='academy:register',key=name_role,value=v_bytes)

# Registeration Form

In [21]:
import cv2
from insightface.app import FaceAnalysis

In [22]:
# configure model
faceapp=FaceAnalysis(name='buffalo_sc',
                     root='insightface_model',
                     providers=['CPUExecutionProvider'])
faceapp.prepare(ctx_id=0,det_size=(640,640),det_thresh=0.5)
# don't set threshold below 0.3

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


## 1. Collect Person and Role

In [ ]:
person=input("Enter your name: ")
role=input("""Please choose:
1. Student
2. Teacher
Enter either 1 or 2.
""")
role
if role=='1':
    role='Student'
else:
    role='Teacher'